In [1]:
%load_ext autoreload
%autoreload 2

In [57]:
import os, sys

import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F

import yaml
import scipy
import numpy as np
import pandas as pd

proj_root = os.path.dirname(os.path.dirname(os.path.abspath(".")))
sys.path.append(proj_root)

from minatar import Environment

from minatar_dqn.my_dqn import Conv_QNET
from minatar_dqn.redo import apply_redo_parametrization

from experiments.experiment_utils import (
    seed_everything,
    search_files_containing_string,
    split_path_at_substring,
    collect_training_output_files,
)

In [3]:
# Collect all paths to models in a specified folder
file_dir = os.path.dirname(os.path.abspath("."))
training_outputs_folder_path = os.path.join(proj_root, "experiments", "training", "outputs")
pruning_outputs_folder_path = os.path.join(file_dir, "outputs")
training_timestamp_folder = "2023_05_15-18_23_40"

experiments_folder = os.path.join(training_outputs_folder_path, training_timestamp_folder)


In [7]:
experiment_paths = collect_training_output_files(
        os.path.join(training_outputs_folder_path, training_timestamp_folder)
    )

for exp in experiment_paths:
    exp["models_folder_path"] = os.path.dirname(exp["model_path"])

In [8]:
experiment_paths

[{'model_path': 'd:\\Work\\PhD\\minatar_work\\experiments\\training\\outputs\\2023_05_15-18_23_40\\conv16_lin64\\breakout\\3\\model_checkpoints\\mck_20',
  'training_folder_path': 'd:\\Work\\PhD\\minatar_work\\experiments\\training\\outputs\\2023_05_15-18_23_40\\conv16_lin64\\breakout\\3',
  'config_path': 'd:\\Work\\PhD\\minatar_work\\experiments\\training\\outputs\\2023_05_15-18_23_40\\conv16_lin64\\breakout\\3\\conv16_lin64_breakout_3_config',
  'stats_path': 'd:\\Work\\PhD\\minatar_work\\experiments\\training\\outputs\\2023_05_15-18_23_40\\conv16_lin64\\breakout\\3\\conv16_lin64_breakout_3_train_stats',
  'models_folder_path': 'd:\\Work\\PhD\\minatar_work\\experiments\\training\\outputs\\2023_05_15-18_23_40\\conv16_lin64\\breakout\\3\\model_checkpoints'},
 {'model_path': 'd:\\Work\\PhD\\minatar_work\\experiments\\training\\outputs\\2023_05_15-18_23_40\\conv32_lin128\\breakout\\3\\model_checkpoints\\mck_20',
  'training_folder_path': 'd:\\Work\\PhD\\minatar_work\\experiments\\traini

For each checkpoint of an experiment we want to prune with different pruning factors and compare against different thresholds + redo scores

In [17]:
exp_paths = experiment_paths[0]

exp_paths["stats_path"]

training_stats_data = torch.load(exp_paths["stats_path"])
redo_scores = training_stats_data["redo_scores"]["policy"]

len(redo_scores)

21

In [24]:
checkpoints_paths = search_files_containing_string(
    exp_paths["models_folder_path"], "mck", substring_location="containing"
)
checkpoints_paths

['d:\\Work\\PhD\\minatar_work\\experiments\\training\\outputs\\2023_05_15-18_23_40\\conv16_lin64\\breakout\\3\\model_checkpoints\\mck_1',
 'd:\\Work\\PhD\\minatar_work\\experiments\\training\\outputs\\2023_05_15-18_23_40\\conv16_lin64\\breakout\\3\\model_checkpoints\\mck_10',
 'd:\\Work\\PhD\\minatar_work\\experiments\\training\\outputs\\2023_05_15-18_23_40\\conv16_lin64\\breakout\\3\\model_checkpoints\\mck_11',
 'd:\\Work\\PhD\\minatar_work\\experiments\\training\\outputs\\2023_05_15-18_23_40\\conv16_lin64\\breakout\\3\\model_checkpoints\\mck_12',
 'd:\\Work\\PhD\\minatar_work\\experiments\\training\\outputs\\2023_05_15-18_23_40\\conv16_lin64\\breakout\\3\\model_checkpoints\\mck_13',
 'd:\\Work\\PhD\\minatar_work\\experiments\\training\\outputs\\2023_05_15-18_23_40\\conv16_lin64\\breakout\\3\\model_checkpoints\\mck_14',
 'd:\\Work\\PhD\\minatar_work\\experiments\\training\\outputs\\2023_05_15-18_23_40\\conv16_lin64\\breakout\\3\\model_checkpoints\\mck_15',
 'd:\\Work\\PhD\\minatar_wor

In [53]:
redo_scores[19]

[tensor([0.0326, 0.0280, 0.0504, 0.0595, 0.0562, 0.0483, 0.0714, 0.0667, 0.0425,
         0.0880, 0.0984, 0.0570, 0.0631, 0.0832, 0.1093, 0.0455]),
 tensor([0.0761, 0.0547, 0.0495, 0.0769, 0.0566, 0.0865, 0.0752, 0.0657, 0.0569,
         0.0691, 0.0589, 0.0425, 0.0630, 0.0495, 0.0692, 0.0498]),
 tensor([1.4707e-03, 6.4590e-02, 1.1963e-02, 1.9946e-03, 1.9096e-03, 2.7781e-04,
         3.6924e-02, 1.4560e-03, 6.9540e-02, 7.8125e-02, 5.0273e-04, 8.9667e-03,
         8.9484e-02, 5.6402e-04, 2.3150e-03, 1.3857e-02, 2.6151e-03, 2.7216e-03,
         3.2223e-02, 3.0299e-03, 0.0000e+00, 9.4393e-03, 3.4041e-03, 0.0000e+00,
         5.2601e-02, 1.3624e-02, 0.0000e+00, 4.2704e-03, 1.1238e-02, 2.9081e-03,
         1.7461e-03, 5.2460e-03, 8.2055e-02, 7.4827e-03, 1.5347e-02, 5.8933e-02,
         2.0834e-02, 6.9320e-07, 2.0289e-03, 1.8741e-03, 2.8097e-03, 3.2972e-05,
         2.9016e-03, 1.3664e-02, 5.2795e-04, 2.3168e-03, 3.1712e-03, 6.3628e-04,
         1.4953e-02, 1.8250e-02, 1.5099e-03, 8.6197e-05,

In [71]:
# read the config so that the model architecture can be loaded
results_df = pd.DataFrame(columns=["checkpoint", "epoch", "layer", "kendall_corr", "pvalue"])

with open(exp_paths["config_path"], "r") as ymlfile:
    cfg = yaml.safe_load(ymlfile)

for check_index, checkpoint_path in enumerate(checkpoints_paths):
    # Get the state of the model
    checkpoint_models_state = torch.load(checkpoint_path)
    checkpoint_model = Conv_QNET(in_features = (4, 10, 10), 
                                in_channels = 4, 
                                num_actions = 6, 
                                **cfg["estimator"]["args_"])
    checkpoint_model = apply_redo_parametrization(checkpoint_model, tau=0.1)
    checkpoint_model.load_state_dict(checkpoint_models_state["policy_model_state_dict"])
    model_state = checkpoint_model.state_dict()

    # Compute kendall rank correlation coefficient for each layer
    layer_weight_keys = ["features.0.weight", "features.2.weight", "fc.0.weight"]
    for i, layer_key in enumerate(layer_weight_keys):
        weights = model_state[layer_key]

        # compute the weight of the neurons in this layer
        if weights.dim() > 2:
            l1_norms = torch.sum(torch.abs(weights), dim=(1, 2, 3))
        else:
            l1_norms = torch.sum(torch.abs(weights), dim=(1))

        neuron_rankings = torch.argsort(l1_norms)
        redo_rankings = torch.argsort(redo_scores[check_index][i])

        # Calculate the Kendall rank correlation coefficient and p-value
        corr, pvalue = scipy.stats.kendalltau(neuron_rankings, redo_rankings)

        new_row = {"checkpoint": os.path.basename(checkpoint_path),
                    "epoch": check_index, 
                    "layer": layer_key,
                    "kendall_corr": corr, 
                    "pvalue": pvalue}
        results_df = pd.concat([results_df, pd.DataFrame(new_row, index=[0])])
        

Hooking Conv2d(4, 16, kernel_size=(3, 3), stride=(1, 1)) -> Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
Hooking Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1)) -> Linear(in_features=576, out_features=64, bias=True)
Hooking Linear(in_features=576, out_features=64, bias=True) -> Linear(in_features=64, out_features=6, bias=True)
Hooking Conv2d(4, 16, kernel_size=(3, 3), stride=(1, 1)) -> Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
Hooking Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1)) -> Linear(in_features=576, out_features=64, bias=True)
Hooking Linear(in_features=576, out_features=64, bias=True) -> Linear(in_features=64, out_features=6, bias=True)
Hooking Conv2d(4, 16, kernel_size=(3, 3), stride=(1, 1)) -> Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
Hooking Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1)) -> Linear(in_features=576, out_features=64, bias=True)
Hooking Linear(in_features=576, out_features=64, bias=True) -> Linear(in_features=64, out_features=6, bia

In [67]:
neuron_rankings.shape

torch.Size([64, 576])

In [68]:
redo_rankings

tensor([ 0, 48,  2, 54,  4,  5, 51,  7, 52, 53, 10, 42, 41, 13, 14, 15, 38, 37,
        57, 58, 20, 60, 62, 23, 40, 25, 26, 29, 30, 44, 16, 36, 61,  3, 50, 34,
        33, 21, 27, 31, 63, 28, 46, 49, 47, 39, 22, 17, 43,  6, 45, 19, 11, 18,
        56, 59, 35,  1, 24,  8,  9, 55, 32, 12])

In [51]:
pvalue

0.8943512599699821

In [ ]:
import pandas as pd

# Create an empty DataFrame with the desired column names
results_df = pd.DataFrame(columns=['experiment', 'accuracy'])

# Run experiments and append results to the DataFrame
for i in range(10):
    # Run experiment and get accuracy
    accuracy = run_experiment(i)

    # Append results to the DataFrame
    new_row = {'experiment': i, 'accuracy': accuracy}
    results_df = pd.concat([results_df, pd.DataFrame(new_row, index=[0])])

# Print the final results DataFrame
print(results_df)
